In [1]:
from queue import Queue
from rtai.utils.config import YamlLoader
from rtai.llm.llm_client import LLMClient
from rtai.agent.agent_manager import AgentManager
from rtai.agent.agent import Agent
from rtai.world.world import World
from rtai.world.clock import clock

cfg = YamlLoader.load("configs/rtai_neil.yaml")
timer = clock(cfg.expand('Clock'))
event_queue = Queue()

world = World(cfg.expand('World'), event_queue)

llm_client = LLMClient()
llm_client.initialize(cfg.expand('LLMClient'))

agent_mgr = AgentManager(event_queue, cfg.expand('Agents'), client=llm_client, world=world)
agent_mgr.initialize()

mark: Agent = agent_mgr.agents['Mark Turner']
sarah: Agent = agent_mgr.agents['Sarah Reynolds']

/opt/homebrew/Caskroom/miniforge/base/envs/embeddings_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sarah_facts = ["Sarah witnessed a heated argument between Emily and Richard Thornton during the social gathering.",
"She noticed Emily and Richard's strained relationship in the weeks leading up to the event.",
"Sarah is aware of financial difficulties in the Thornton marriage, with Richard contemplating a divorce.",
"During the argument, Sarah overheard Emily expressing frustration about Richard's intention to file for divorce.",
"Sarah is acquainted with other guests at the event who had their own reasons to dislike Richard Thornton."]

mark_facts = [
    "Richard had business rivals who might have had motives to harm him.",
    "There is a potential alibi for Emily during the time of the poisoning, involving a witness not yet presented in court.",
    "Inconsistencies exist in the initial police investigation, including gaps in the timeline and overlooked potential suspects.",
    "Richard was involved in a secret business deal that might have contributed to his strained relationships.",
    "Mark possesses an anonymous letter received by the Thorntons, hinting at threats from an unknown source."
]

In [3]:
for fact in sarah_facts:
    sarah.l_mem.add_concept(fact)

for fact in mark_facts:
    mark.l_mem.add_concept(fact)

In [4]:
mark.l_mem.create_embeddings()
sarah.l_mem.create_embeddings()

In [5]:
sarah.l_mem.retriever.retrieve_context("How was Emily and Richard's relationship leading up to the murder")

text
recency score for She noticed Emily and Richard's strained relationship in the weeks leading up to the event. is 10.0
recency score for During the argument, Sarah overheard Emily expressing frustration about Richard's intention to file for divorce. is 10.0
recency score for Sarah witnessed a heated argument between Emily and Richard Thornton during the social gathering. is 10.0
recency score for Sarah is aware of financial difficulties in the Thornton marriage, with Richard contemplating a divorce. is 10.0
recency score for Sarah is acquainted with other guests at the event who had their own reasons to dislike Richard Thornton. is 10.0
raw score for all retrieved is [107.46542072296143, 107.2093636393547, 107.0806303024292, 106.8001412153244, 106.72763621807098]
normalized score for all retrieved is [1.0, 0.652937840372856, 0.4784514746763807, 0.09827394960563, 0.0]


"She noticed Emily and Richard's strained relationship in the weeks leading up to the event.During the argument, Sarah overheard Emily expressing frustration about Richard's intention to file for divorce.Sarah witnessed a heated argument between Emily and Richard Thornton during the social gathering."

In [8]:
prompt = "You are a story teller telling a drama about a court case"
dialogue_topic = "Mark is helping Sarah prepare for taking the stand"
location = "Mark's office"

In [9]:
import guidance
from guidance import models
from guidance import gen, select


@guidance
def generate_dialogue(lm, agent_name, persona, topic, ctx, chat_history):
    lm += f'''
    You are {agent_name}, Here is your personality: {persona}
    
    Here is the dialogue:
    {chat_history}

    Here is the context that {agent_name} knows: {ctx}

    now, continue the dialogue with ONE short line as if you are {agent_name}. Incorporate their personality into the response
    {agent_name}: {gen(name="convo", temperature=0.8, max_tokens=1000, stop=".")}
    '''
    return lm

mistral = models.LlamaCpp("/Users/nyeung/Projects/llama.cpp/models/mistral-7b-instruct-v0.2.Q4_K_M.gguf", n_gpu_layers=-1, n_ctx=10000)
chat_history = ["Sarah Turner: I heard the prosecutor is tough. Really tough."]
thought_history = []
lm = mistral + "You are a story teller telling a drama about a court case. Make sure the dialogue is dramatic, but consistent with what each person knows."

ggml_metal_free: deallocating


In [17]:
# Run chat
def run_chat(agent1: Agent, agent2: Agent):
    for i in range(3):
        if i % 2 == 0:
            agent_name = agent1.get_name()
            persona = agent1.get_common_set_str()
            ctx = agent1.l_mem.retriever.retrieve_context(chat_history[-1])
        else:
            agent_name = agent2.get_name()
            persona = agent2.get_common_set_str()
            ctx = agent2.l_mem.retriever.retrieve_context(chat_history[-1])
        print(ctx)
        out = lm + generate_dialogue(agent_name=agent_name, persona=persona, topic=dialogue_topic, ctx=ctx, chat_history=chat_history)
        chat_history.append(f"{agent_name}: " + out["convo"])
        out.reset()
        print(out["convo"])

In [11]:
mark.l_mem.id_to_node

{0: Richard had business rivals who might have had motives to harm him.,
 1: There is a potential alibi for Emily during the time of the poisoning, involving a witness not yet presented in court.,
 2: Inconsistencies exist in the initial police investigation, including gaps in the timeline and overlooked potential suspects.,
 3: Richard was involved in a secret business deal that might have contributed to his strained relationships.,
 4: Mark possesses an anonymous letter received by the Thorntons, hinting at threats from an unknown source.}

In [18]:
run_chat(mark, sarah)

None


In [19]:
chat_history

['Sarah Turner: I heard the prosecutor is tough. Really tough.',
 "Mark Turner: 'Sarah, my client's innocence is not just about convenience",
 'Sarah Reynolds: "Mark, I can\'t ignore the truth, but I\'m not ready to betray a friend\'s trust',
 "Mark Turner: 'Sarah, trust me",
 'Sarah Reynolds: "Mark, I know Emily\'s frustration, but she didn\'t deserve to lose her husband this way',
 "Mark Turner: 'Sarah, let's focus on the facts and the truth, not emotions or convenience",
 'Sarah Reynolds: "Mark, I know Emily\'s frustration, but I can\'t forget Richard\'s indiscreet words that night',
 "Mark Turner: 'Sarah, let's use the potential alibi and the compelling facts to question the prosecution's case, and uncover the hidden truth behind the poisoning'",
 'Sarah Reynolds: "Mark, I understand Emily\'s pain, but I also know Richard\'s indiscreet ways could\'ve led to this unfortunate outcome',
 "Mark Turner: 'Sarah, let's dig deeper, uncover the truth and reveal the real motives behind Richa

In [15]:
import pandas as pd

a = pd.read_csv("output.csv")
print(a)

              speaker                                               line
0     Claire Reynolds   Ladies and Gentleman of the Jury, the Prosecu...
1   Isabella Martinez   I swear on my childrens lives that I did not ...
2     Claire Reynolds                                        Prosecution
3   Isabella Martinez   I was at the grocery store, buying milk for m...
4     Claire Reynolds                                        Prosecution
5   Isabella Martinez   My neighbor was shopping for me and can vouch...
6     Claire Reynolds                                        Prosecution
7   Isabella Martinez   I trust my neighbor will confirm my presence ...
8     Claire Reynolds   Well be in touch with your neighbor, Ms. Mart...
9   Isabella Martinez   I have faith in the truth and trust the justi...
10    Claire Reynolds   Please remember, the burden of proof is on th...


In [16]:
string_rep = ', '.join(f"{speaker}: {line}" for speaker, line in zip(a['speaker'], a['line']))

In [17]:
string_rep

'Claire Reynolds:  Ladies and Gentleman of the Jury, the Prosecution calls Isabella Martinez to the stand., Isabella Martinez:  I swear on my childrens lives that I did not steal the necklace., Claire Reynolds:  Prosecution, Isabella Martinez:  I was at the grocery store, buying milk for my sick son., Claire Reynolds:  Prosecution, Isabella Martinez:  My neighbor was shopping for me and can vouch for my whereabouts., Claire Reynolds:  Prosecution, Isabella Martinez:  I trust my neighbor will confirm my presence at the grocery store., Claire Reynolds:  Well be in touch with your neighbor, Ms. Martinez. Your testimony will be crucial to the verdict., Isabella Martinez:  I have faith in the truth and trust the justice system will see my innocence., Claire Reynolds:  Please remember, the burden of proof is on the Prosecution. Well update you on the neighbors verification.'

In [18]:
@guidance
def generate_story(lm, dialogue):
    lm += f'''
    You are a story teller telling a drama about a court case. Create a short compelling story that is interesting and dramatic. Incorporate the dialogue into the story.
    
    Here is the Dialogue:
    {dialogue}

    {gen(name="story", temperature=0.8, max_tokens=2000)}
    '''
    return lm
lm + generate_story(dialogue=string_rep)

In [61]:
prompt = f"""
Generate a short dialogue between Sarah and Mark in {location}

Sarah's persona: {sarah.get_common_set_str()}
Sarah's context: {sarah.l_mem.retriever.retrieve_context(dialogue_topic)}
Mark's persona: {mark.get_common_set_str()}
Mark's context: {sarah.l_mem.retriever.retrieve_context(dialogue_topic)}

Here is the short dialogue:
{gen('dialogue', max_tokens=1000)}"""

story = mistral + "You are a master Hollywood scriptwriter who specializes in legal dramas. Conduct each task as if you are that scriptwriter."
out3 = story + prompt
dialogue = out3["dialogue"]
out3.reset()

In [68]:
clean_dialogue = dialogue.split("\n")

for line in clean_dialogue:
    if line == '':
        continue
    else:
        print(line.split(":", maxsplit=1)[1])

 (leaning back in his chair, studying Sarah intently) Sarah, I know you've been through a lot since that night. I want to understand your perspective on what you saw and heard.
 (nervously fidgeting with her hands) Mark, I don't want to get involved in this. I mean, I'm just a bystander. I was there when it happened, but I didn't see anything definitive.
 (leaning forward, his tone softening) Sarah, your presence at the scene is significant. You've known the Thorntons for years, and your testimony could help us build a stronger defense for Emily. I understand your reluctance, but I believe that the truth will ultimately prevail.
 (hesitant) Mark, I don't want to betray a friend. I know Emily is going through a difficult time, and I don't want to add to her pain. But I also can't ignore what I heard that night.
 (nodding, understandingly) Sarah, I respect your loyalty, and I understand your concerns. But the truth is, the prosecution will use your presence at the scene against us. They'

In [53]:
from guidance import select

@guidance
def reflect(lm, name, dialogue):
    lm += f'''What are the top 3 takeaways for {name} from this dialogue: {dialogue}?
    {gen(name="takeaways", temperature=0.8, max_tokens=300)}'''

    lm += f'''
    Based off those three takwaways, what should {name} do next?

    Your options are EAT: go eat food, CHAT: go chat with another witness, CRY: go cry in the bathroom
    {select(name="do_next", options=["SOB", "CHAT", "CRY"])} because {gen(name="reason", temperature=0.8, max_tokens=300)}'''

    if lm["do_next"] == "CHAT":
        lm += f'''Q: Who should {name} chat with next?
        A: {select(name="next_chat", options=["Sarah", "Emily", "Richard"])}''' 
    return lm

@guidance
def act(lm, name, options):
    lm += f'''What should {name} do next?
    {select(name="do_next", options=options)}'''
    return lm

In [55]:
out4 = lm + reflect(name="Mark", dialogue=dialogue)

In [56]:
out4["next_chat"]

'Emily'

In [29]:
dialogue.split("\n")

['',
 '---',
 '',
 "(Mark's office, Sarah sits nervously in a chair, her hands fidgeting with a pen. Mark, with a calm demeanor, sits behind his desk.)",
 '',
 "Mark: Sarah, I understand that this is a difficult situation for you. But I need to know everything you've observed that night.",
 '',
 "Sarah: (pauses, looks down) I saw Emily and Richard arguing. They were in the kitchen, and I couldn't make out what they were saying, but it was heated.",
 '',
 'Mark: And what about the other guests? Were there any suspicious interactions?',
 '',
 "Sarah: (hesitates) There were a few guests who seemed to have a reason to dislike Richard. But I can't be sure if they had anything to do with his poisoning.",
 '',
 'Mark: (leaning forward) Sarah, I need you to be honest with me. Did you notice anything that could potentially implicate Emily?',
 '',
 "Sarah: (sighs) I did overhear Emily expressing frustration about Richard's plans to file for divorce. But I don't think she would actually harm him.

In [7]:
# throw the chat history into a dataframe
import re
import pandas as pd
speakers = []
lines = []
for line in chat_history:
    try: 
        new_line = line.split(":")
        speakers.append(new_line[0])
        lines.append(re.sub(r'[\'"]', '', new_line[1])) # clean quotes
    except:
        pass

In [13]:
df = pd.DataFrame({"speaker": speakers, "line": lines})

In [14]:
df.to_csv("output.csv", index=False)

In [28]:
lm["convo"]

In [17]:
speakers = ["Isabella Martinez", "Claire Reynolds", "Isabella Martinez", "Claire Reynolds", "Isabella Martinez"]
lines = [lm[f"convo{i}"] for i in range(1, 6)]

for line in lines:
    try:
        print(line.split(":")[1])
    except:
        print(line)

(enters Claire's office, looking relaxed but focused) Claire, I have some updates on the Hernandez case
(looking up from her paperwork) Yes, Isabella
 (leaning back in her chair) That's significant progress
 What about other evidence? Any new leads on the weapon or the motive?
Isabella Martinez
 (nods thoughtfully) That's important


In [15]:
lines

["(enters Claire's office, looking relaxed but focused) Claire, I have some updates on the Hernandez case",
 '(looking up from her paperwork) Yes, Isabella',
 "The forensic analysis on the victim's clothing came back, and there's a definite match to the tattoo on the suspected killer's hand\nClaire Reynolds: (leaning back in her chair) That's significant progress",
 ": What about other evidence? Any new leads on the weapon or the motive?\nIsabella Martinez: Not yet on the weapon, but we're still working on it",
 "As for the motive, the victim's husband mentioned some financial troubles and infidelity\nClaire Reynolds: (nods thoughtfully) That's important"]

In [25]:
import pandas as pd

df = pd.DataFrame(columns=['speaker', 'dialogue'])

"I've been looking into the victim's background and I've discovered some interesting information\nClaire Reynolds: Oh, really? Do share\nIsabella Martinez: The victim had some financial troubles in the past"

# Archive

In [ ]:
def pretty_print_schedule(name, daily_schedule):
    print(f"{name}'s Daily Schedule:")
    print("\n")
    for task in daily_schedule:
        print(task[0].split('\"')[0] + " for " + str(task[1]) + " hours starting at " + str(task[2]))

pretty_print_schedule(agent1.get_name(), agent1.s_mem.daily_schedule)
print("\n")
pretty_print_schedule(agent2.get_name(), agent2.s_mem.daily_schedule)

In [ ]:
import guidance
from guidance import models
from guidance import gen, select

@guidance
def generate_dialogue(lm, agent_name, topic, chat_history):
    lm += f'''
    Here is the Chat History:
    {chat_history}
    
    continue the chat about {topic} with ONE short line as if you are {agent_name}
    {agent_name}: {gen(stop="'", name="convo", temperature=0.7)}
    '''
    return lm

@guidance
def evaluate_chat(lm, chat_history):
    lm += f'''
    You are facilitating a roleplay between two people. Your objective is to faciliate an interesting dramatic dialgoue and prevent the dialogue from getting stale. If the dialogue starts repeating, choose to end the roleplay.
    
    Decide whether to continue the roleplay or not.

    Here is the Roleplay so far:
    {chat_history}
    
    Now you will choose whether to either CONTINUE or END the roleplay.
    Choice: {select(name="evaluate_chat", options=["CONTINUE", "END"])}
    '''
    return lm

'''
A function for two agents to chat about a topic for a certain number of iterations or until the chat is getting stale
'''
def chat(agent1, agent2, topic, max_iters=5):
    models.LlamaCpp("/Users/nyeung/Projects/llama.cpp/models/mistral-7b-instruct-v0.2.Q4_K_M.gguf", n_gpu_layemistral = rs=-1, n_ctx=4000)
    # mistral.echo = False
    lm = mistral + "You are a story teller telling a drama about a court case"
    iters = 0
    chat_history = []
    end_chat = False

    # chatting loop
    while iters < max_iters and not end_chat:
        print("Running iter %d" % iters)
        if iters % 2 == 0:
            agent = agent1
        else:
            agent = agent2
        out1 = lm + generate_dialogue(agent, topic, chat_history)
        chat_history.append(f"{agent1}: " + out1["convo"])
        out2 = lm + evaluate_chat(chat_history)

        # if lm says to end the chat, then we end
        if out2["evaluate_chat"] == "END":
            print("Deciding to end chat")
            end_chat = True
        
        iters += 1
    
    return chat_history

In [ ]:
import guidance
from guidance import gen

@guidance
def generate_dialogue(lm, agent1_name, chat_ctx):
    lm += f'''
    Generate a conversation between {agent1_name} and {agent2_name} about {dialogue_topic} set in {location}.
    Here is the dialogue:
    {agent1_name}: {gen(stop=".", name="convo1", temperature=1.0)}
    {agent2_name}: {gen(stop=".", name="convo2", temperature=1.0)}
    {agent1_name}: {gen(stop=".", name="convo3", temperature=1.0)}
    {agent2_name} {gen(stop=".", name="convo4", temperature=1.0)}
    {agent1_name}: {gen(stop=".", name="convo5", temperature=1.0)}
    '''
    lm.set("convo", lm["convo1"] + lm["convo2"] + lm["convo3"] + lm["convo4"] + lm["convo5"])
    return lm